## IoT 2021 - Assignment #2

#### Radio settings

If you are running this training at the same time as other people on the testbed, it is a good idea to change the default radio configuration to avoid too much collision with others.

Use the following cell to give you random values for channel and PAN ID.

In [ ]:
%env SITE=saclay

In [ ]:
!whoami

In [ ]:
import os,binascii,random
pan_id = binascii.b2a_hex(os.urandom(2)).decode()
channel = random.randint(11, 26)
print('Use CHANNEL={}, PAN_ID=0x{}'.format(channel, pan_id))

In [ ]:
print(channel)

In [ ]:
%env CHANNEL=22

In [ ]:
%env PAN_ID=0xc1fc

## Submit an experiment on IoT-LAB

We reserve the following nodes:
* 1 a8 node that will act as MQTT/SN broker and Transparent Bridge to the IoT Core
* 1 m3 node that will act as border router
* 2+ m3 nodes that will run the application

In [ ]:
!iotlab-experiment submit -n riot_m3 -d 120 -l 4,archi=m3:at86rf231+site=$SITE -l 1,archi=a8:at86rf231+site=$SITE

In [ ]:
!iotlab-experiment wait --timeout 60 --cancel-on-timeout

Get basic info about the reserved nodes:

In [ ]:
!iotlab-experiment --jmespath="items[*].network_address | sort(@)" get --nodes

Use the following commands for additional info about the reserved nodes:

In [ ]:
!iotlab-experiment get -i <EXPERIMENT_ID> -p

In [ ]:
!iotlab-experiment get -i <EXPERIMENT_ID> -n

### Open a terminal to build and flash the firmware on the iotlab-m3

1. Open a Jupyter terminal (use `File > New > Terminal`) and run the following commands from it.

The following command needs to be run ONLY once

### We will use 3 distrinct terminals T\#{1,2,3}

#### T\#1 iotlab-m3 (Border Router)
Execute the following commands, by replacing the \<PLACEHOLDERS> with real info:

`source /opt/riot.source`

`cd RIOT`

`make ETHOS_BAUDRATE=500000 DEFAULT_CHANNEL=<CHANNEL> BOARD=iotlab-m3 -C examples/gnrc_border_router clean all`

`iotlab-node --flash examples/gnrc_border_router/bin/iotlab-m3/gnrc_border_router.elf -l saclay,m3,<ID>`

`sudo ethos_uhcpd.py m3-<ID> tap<K> 2001:660:3207:04c1::1/64`

#### T\#2 iotlab-a8 (MQTT broker)
Execute the following commands:

From your local machine, use the following command from the /etc/mosquitto/ directory:

`scp -r certs/ <login>@saclay.iot-lab.info:~/A8`

Create the following files inside the A8 folder:

1) config.conf

2) mosquitto.conf

3) bridge.conf

`ssh root@node-a8-<ID>`


`cd A8`

`cp -r certs /etc/mosquitto && cp mosquitto.conf /etc/mosquitto && mkdir /etc/mosquitto/conf.d && cp bridge.conf /etc/mosquitto/conf.d`

`mkdir /var/log/mosquitto && touch /var/log/mosquitto/mosquitto.log && chmod 666 /var/log/mosquitto/mosquitto.log`

`/usr/sbin/mosquitto -c /etc/mosquitto/mosquitto.conf &`

`ip -6 -o addr show eth0` // TO GET THE IP Address of the MQTT broker

`broker_mqtts config.conf`

#### T\#3 iotlab-m3 (IoT application)
For all the remaining reserved m3 nodes, execute the following commands:

`source /opt/riot.source`

`cd IoT-2021-Individual-Assigments/dev/`

`make DEFAULT_CHANNEL=<CHANNEL> SERVER_ADDR=<MQTT_SERVER_ADDR> EMCUTE_ID=power_saver<K> BOARD=iotlab-m3 -C . clean all`

`iotlab-node --flash ./bin/iotlab-m3/assignment2.bin -l saclay,m3,<ID>`

Eventually, nc to the node\[s\] to see if everything works fine:

`nc m3-<ID> 20000`

### Free up the resources

Once done, stop your experiment to free up the experiment nodes:

In [ ]:
!iotlab-experiment stop

## Performance Evaluation

In [ ]:
!unzip perf.zip

In [ ]:
!ls perf/tst3/266481/consumption

In [ ]:
import pandas

In [ ]:
dataset = pandas.read_csv("./perf/tst3/266481/consumption/m3_12.oml", sep='\t', skiprows=9, header=0, names=["timestamp_s", "timestamp_us", 'ign1', 'ign2', "power", "voltage", "current"])

In [ ]:
dataset

In [ ]:
ax = dataset.power.plot(figsize=(14,4), label='Power (W)')
ax.set_ylabel('Power (W)')
ax.set_xlabel('Time (ms)')

In [ ]:
ax = dataset.current.plot(figsize=(14,4), label='Current (mAh)')
ax.set_ylabel('Current (mAh)')
ax.set_xlabel('Time (ms)')

In [ ]:
ax = dataset.voltage.plot(figsize=(14,4), label='Voltage (V)')
ax.set_ylabel('Voltage (V)')
ax.set_xlabel('Time (ms)')

In [ ]:
!ls perf/tst1/266474/radio

In [ ]:
dataset2 = pandas.read_csv("./perf/tst3/266481/radio/m3_12.oml", sep='\t', skiprows=9, header=0, names=["timestamp_s", "timestamp_us", "channel", "rssi"])

In [ ]:
dataset2

In [ ]:
ax = dataset2.rssi.plot(figsize=(14,4), label='RSSI (dBm)')
ax.set_ylabel('RSSI (dBm)')
